<a href="https://colab.research.google.com/github/jimregan/wolnelektury-audio-corpus/blob/new-vocab/poleval_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers -q

     |████████████████████████████████| 778kB 8.3MB/s 
     |████████████████████████████████| 3.0MB 24.2MB/s 
     |████████████████████████████████| 1.1MB 52.2MB/s 
     |████████████████████████████████| 890kB 51.9MB/s 


In [2]:
from transformers import *

In [3]:
model = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
tokenizer = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
nlp = pipeline('fill-mask', model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at dkleczek/bert-base-polish-uncased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
!wget http://poleval.pl/task1/ClarinPlTrain.zip http://poleval.pl/task1/ClarinPlTest.zip http://poleval.pl/task1/ClarinPlDev.zip http://poleval.pl/task1/Sejm.zip

--2020-08-19 09:31:55--  http://poleval.pl/task1/ClarinPlTrain.zip
Resolving poleval.pl (poleval.pl)... 213.135.36.94
Connecting to poleval.pl (poleval.pl)|213.135.36.94|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40044998 (38M) [application/zip]
Saving to: ‘ClarinPlTrain.zip’

ClarinPlTrain.zip   100%[===================>]  38.19M  50.0MB/s    in 0.8s    

2020-08-19 09:31:56 (50.0 MB/s) - ‘ClarinPlTrain.zip’ saved [40044998/40044998]

--2020-08-19 09:31:56--  http://poleval.pl/task1/ClarinPlTest.zip
Reusing existing connection to poleval.pl:80.
HTTP request sent, awaiting response... 200 OK
Length: 5041323 (4.8M) [application/zip]
Saving to: ‘ClarinPlTest.zip’

ClarinPlTest.zip    100%[===================>]   4.81M  31.3MB/s    in 0.2s    

2020-08-19 09:31:56 (31.3 MB/s) - ‘ClarinPlTest.zip’ saved [5041323/5041323]

--2020-08-19 09:31:56--  http://poleval.pl/task1/ClarinPlDev.zip
Reusing existing connection to poleval.pl:80.
HTTP request sent, awaiting r

In [6]:
!for i in *.zip;do b=$(basename $i '.zip'); mkdir $b;pushd $b; unzip ../$b.zip; popd;done

/content/ClarinPlDev /content
Archive:  ../ClarinPlDev.zip
  inflating: 1best.txt               
  inflating: lat.txt                 
  inflating: nbest.txt               
  inflating: reference.txt           
/content
/content/ClarinPlTest /content
Archive:  ../ClarinPlTest.zip
  inflating: 1best.txt               
  inflating: lat.txt                 
  inflating: nbest.txt               
  inflating: reference.txt           
/content
/content/ClarinPlTrain /content
Archive:  ../ClarinPlTrain.zip
  inflating: 1best.txt               
  inflating: lat.txt                 
  inflating: nbest.txt               
  inflating: reference.txt           
/content
mkdir: cannot create directory ‘Sejm’: File exists
/content/Sejm /content
Archive:  ../Sejm.zip
   creating: Sejm/
  inflating: Sejm/1best.txt          
  inflating: Sejm/lat.txt            
  inflating: Sejm/reference.txt      
  inflating: Sejm/nbest.txt          
/content


In [7]:
!apt install wdiff

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
Suggested packages:
  wdiff-doc
The following NEW packages will be installed:
  wdiff
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 29.6 kB of archives.
After this operation, 119 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 wdiff amd64 1.2.2-2 [29.6 kB]
Fetched 29.6 kB in 0s (509 kB/s)
Selecting previously unselected package wdiff.
(Reading database ... 144487 files and directories currently installed.)
Preparing to unpack .../wdiff_1.2.2-2_amd64.deb ...
Unpacking wdiff (1.2.2-2) ...
Setting up wdiff (1.2.2-2) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [13]:
import os
os.chdir('/content')

In [16]:
!for i in ClarinPlDev/ ClarinPlTest/ ClarinPlTrain/ Sejm/;do pushd $i; sort 1best.txt > 1best-s.txt; sort reference.txt > ref-s.txt; wdiff 1best-s.txt ref-s.txt > diff.txt; cat diff.txt |perl -C7 -ane 'while(/\pL \[\-(\pL+)\-\] \pL/g) {print "$1\n";}'  > intrusive.txt; cat diff.txt |perl -C7 -ane 'while(/\[\-(\pL[\pL\- ]\pL+)\-\] \{\+(\pL[\pL\- ]\pL+)\+\}/g) {print "$1\t$2\n";}' > pairs.tsv;popd;done

/content/ClarinPlDev /content
/content
/content/ClarinPlTest /content
/content
/content/ClarinPlTrain /content
/content
/content/Sejm /content
/content


In [20]:
!for i in ClarinPlDev/ ClarinPlTest/ ClarinPlTrain/ Sejm/;do cat $i/intrusive.txt;done|sort|uniq -c|awk '($1>4){print $2}' > freq-intrusions.txt

In [22]:
!for i in ClarinPlDev/ ClarinPlTest/ ClarinPlTrain/ Sejm/;do cat $i/pairs.tsv ;done | sort|uniq -c|sort -nr > pairs.tsv
